<div style="background-color: #cfc ; padding: 20px; border-radius: 10px ; border: 2px solid green;">
<p><font size="+3"><b><center> qcware.qMC </center></b></font>
</p>
<p>
<font size="+3">  <b> <center> The Quantum Monte Carlo API</center></b></font>
</p>    
</div>

**qcware.qMC** is Forge's API for Quantum Monte Carlo simulations.

Our ambition is to deliver the first real applications of quantitative finance with Quantum Monte Carlo simulations. The following notebook will show you how to price a European option with quantum computers using Forge.

## Import QCWare libraries

In [1]:
import os,sys,inspect

In [2]:
import quasar
import qcware
import numpy as np
from qcware.forge import qutils
from qcware.forge.montecarlo.nisqAE import make_schedule#, nisqAE, MLE_for_nisqAE
from european_option import EuropeanOption


## Classical parameters for option pricing

In [3]:
# Define classical parameters for the Option pricing

nAssetPrices = 8      # Number of different Asset Prices generated for the probability distribution
S0 = 100              # Initial Asset Price
K = 110               # Strike
r = 0.05              # interest rate
sigma = 0.1           # Volatility
T = 1                 # Time to maturity
optionType = 'C'      # 'C' = European Call Option, 'P' = European Put Option
epsilon = 0.005        # precision parameter for the estimate

## The EuropeanOption class

In [4]:
# Define a new European Option product
newProduct = EuropeanOption(nAssetPrices,S0,K,r,sigma,T,optionType,epsilon)

In [5]:
# Price with Black-Scholes formula
print("Price with Black Scholes formula:",round(newProduct.BSM,3))

Price with Black Scholes formula: 2.174


### Price using the default NISQ Ampitude Estimation algorithm

In [6]:
newProduct.price()

2.0247506459197493

## Going deeper into the quantum Monte Carlo method

Let us provide a high level description of the quantum Monte Carlo method.

We start with the model, which is a quantum circuit which we can measure directly to get a sample. For example here, the quantum circuit creates a distribution over asset prices and then computes the payoff function for each of them in superposition, so that when we measure we get a sample from the distribution of the expected payoff. 

The main idea is that not only do we have the possibility to sample from this quantum circuit directly, but we can create deeper quantum circuits by repeating this quantum model sequentially, so that sampling from all these quantum circuits with different depths and cleverly combining all the results, reduces the total number of samples. 

Thus, the first important part of the quantum Monte Carlo method is to define a schedule of which circuits to samples from and how many times. In other words, we need to define a schedule, which is a list of pairs {($D_1$,$N_1$), ($D_2$,$N_2$),...($D_k$,$N_k$)}, that tells the quantum algorithm to run each quantum circuit of depth $D_i$ for $N_i$ shots. Different schedules give different accuracies with different number of samples!

Here, we have some predefined types of schedules ('linear','exponential','powerlaw,'direct') one can use or one can define their own schedule by just providing as a list.

### Schedules for quantum pricing

We can use different types of schedules for the quantum pricing algorithm.

In [7]:
# Examples of schedules
# (scheduleType='direct', nShots=10000) : [[0,10000]]
# (scheduleType='linear', maxDepth=10,nShots=10):  [[0,10],[1,10],[2,10],[3,10],[4,10],...,[10,10]]
# (scheduleType='exponential', maxDepth=20,nShots=10) [[1,10],[2,10],[4,10],[8,10],[16,10]]
# (scheduleType='powerlaw', beta=0.2, nShots=10) : this is a more complicated schedule that depends on beta
# which takes values from [0.1 , 0.9]. Attention, the exponent beta can result in very large computations


# Schedule parameters are optional. By default, a linear schedule of maxDepth=10 and nShots=10 will be used
schedule = None         # provide a schedule, eg [[0,20],[1,20],[2,20]] or put None for predefined schedule types
scheduleType = 'exponential' # schedule type: 'direct', 'linear','exponential','powerlaw', (only if schedule=None)
nShots = 10             # shots per circuit
maxDepth = 20    # parameter for 'linear' and 'exponential' to define the maximum depth of a circuit
beta = 0.3              # parameter for the 'powerlaw' schedule between [0.1,0.9]

In [8]:
# we can price again with a different schedule
newProduct.price(scheduleType='exponential', maxDepth=16,nShots=10)

1.9413351971712158

In [9]:
# You can print the schedule that was used for the pricing
print(newProduct.schedule)

[[1, 10], [2, 10], [4, 10], [8, 10], [16, 10]]


#### Benchmark different schedules

In [10]:
# benchmarking
# We are going to test how well the different schdules
# approximate the real value and 
# how many samples they use

# 1. real value
realPrice = newProduct.price(nShots=None)
print('Exact price after discretization :',realPrice)

# 2. classical value
classicalPrice = newProduct.price(scheduleType='direct', nShots=100000)
print('Quantum price - direct', classicalPrice)
print('Samples - direct', newProduct.samples)
#print(newProduct.schedule)

# 3. quantum value – linear schedule
quantumPrice=newProduct.price(scheduleType='linear', maxDepth=10,nShots=10)
print('Quantum price - linear', quantumPrice)
print('Samples - linear', newProduct.samples)
#print(newProduct.schedule)

# 3. quantum value – linear schedule
quantumPrice=newProduct.price(scheduleType='exponential', maxDepth=20,nShots=10)
print('Quantum price - exponential', quantumPrice)
print('Samples - linear', newProduct.samples)
#print(newProduct.schedule)

# 4. quantum value – powerlaw schedule beta=beta,
quantumPrice = newProduct.price(scheduleType='powerlaw', nShots=10, beta=0.2)
print('Quantum price - powerlaw', quantumPrice)
print('Samples - powerlaw', newProduct.samples)
#print(newProduct.schedule)


Exact price after discretization : 1.9441536619918698


Quantum price - direct 1.9413351971712158
Samples - direct 100000


Quantum price - linear 1.9413351971712158
Samples - linear 1210


Quantum price - exponential 1.9413351971712158
Samples - linear 670


Quantum price - powerlaw 1.9413351971712158
Samples - powerlaw 5610


### Looking into the quantum circuits

We can check the quantum circuits that correspond to the model (initiak_circuit) and to the iteration circuit sequentially in the quantum Monte Carlo method (iteration_circuit)

In [11]:
print('Initial circuit – Oracle\n')
print(newProduct.initial_circuit)

Initial circuit – Oracle

T  : |0|1|2|3|4 |5|6 |7|8 |9|10|11|12|13|14|15|

q0 : -X-B-B-B----------------------------------
        | | |                                  
q1 : ---|-|-S----------------------------------
        | |                                    
q2 : ---|-S-B----------------------------------
        |   |                                  
q3 : ---|---S----------------------------------
        |                                      
q4 : ---S-B-B----------------------------------
          | |                                  
q5 : -----|-S----@----@------------------------
          |      |    |                        
q6 : -----S-B----|----|----@----@--------------
            |    |    |    |    |              
q7 : -------S----|----|----|----|-----@-----@--
                 |    |    |    |     |     |  
q8 : ---------Ry-X-Ry-X-Ry-X-Ry-X--Ry-X--Ry-X--
                                               
T  : |0|1|2|3|4 |5|6 |7|8 |9|10|11|12|13|14|15|



In [12]:
print('Iteration circuit \n')
print(newProduct.iteration_circuit)

Iteration circuit 

T  : |0|1|2 |3|4 |5|6 |7|8 |9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|

q0 : ----------------------------------B--B--B--@--Z--B--B--B-----------------
                                       |  |  |  |     |  |  |                 
q1 : ----------------------------------S--|--|--|-----|--|--S-----------------
                                          |  |  |     |  |                    
q2 : ----------------------------------B--S--|--|-----|--S--B-----------------
                                       |     |  |     |     |                 
q3 : ----------------------------------S-----|--|-----|-----S-----------------
                                             |  |     |                       
q4 : ----------------------------------B--B--S--|-----S--B--B-----------------
                                       |  |     |        |  |                 
q5 : -----------------------@----@-----S--|-----|--------|--S-----@-----@-----
                            |  

#### Using parallel quantum circuits

We can use a different type of quantum circuits that uses more qubits but swallower circuits

In [13]:
# one optional parameter for optimizing depth vs qubits
mode = 'parallel'   # 'parallel' = optimized depth, 'sequential' = optimized number of qubits

# Define a new European Option product
newProduct_parallel = EuropeanOption(nAssetPrices,S0,K,r,sigma,T,optionType,epsilon,mode=mode)
newProduct_parallel.price()

1.9413351971712158

In [14]:
print(newProduct_parallel.initial_circuit)

T   : |0|1|2|3|4 |5|6 |7|8 |9|10|11|12|13|14|15|

q0  : -X-B-B-B----------------------------------
         | | |                                  
q1  : ---|-|-S----------------------------------
         | |                                    
q2  : ---|-S-B----------------------------------
         |   |                                  
q3  : ---|---S----------------------------------
         |                                      
q4  : ---S-B-B----------------------------------
           | |                                  
q5  : -----|-S----@----@------------------------
           |      |    |                        
q6  : -----S-B----|----|----@----@--------------
             |    |    |    |    |              
q7  : -------S----|----|----|----|-----@-----@--
                  |    |    |    |     |     |  
q8  : ---------Ry-X-Ry-X----|----|-----|-----|--
                            |    |     |     |  
q9  : -------------------Ry-X-Ry-X-----|-----|--
                   

In [15]:
print(newProduct_parallel.iteration_circuit)

T   : |0|1|2 |3|4 |5|6 |7|8 |9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|

q0  : ----------------------------------B--B--B--@--@--@--Z--B--B--B-----------
                                        |  |  |  |  |  |     |  |  |           
q1  : ----------------------------------S--|--|--|--|--|-----|--|--S-----------
                                           |  |  |  |  |     |  |              
q2  : ----------------------------------B--S--|--|--|--|-----|--S--B-----------
                                        |     |  |  |  |     |     |           
q3  : ----------------------------------S-----|--|--|--|-----|-----S-----------
                                              |  |  |  |     |                 
q4  : ----------------------------------B--B--S--|--|--|-----S--B--B-----------
                                        |  |     |  |  |        |  |           
q5  : -----------------------@----@-----S--|-----|--|--|--------|--S-----@-----
                             |    |    

#### Defining your own quantum circuits

You can define your own circuits for applying the quantum Monte Carlo methods. More details on the NISQ Amplitude Estimation notebook